In [1]:
import torch 
from matplotlib import pyplot as plt
import numpy as np 
import cv2
import os 
import uuid
import time
import pandas as pd


In [2]:
torch.cuda.is_available()

True

In [3]:
model=torch.hub.load('ultralytics/yolov5','yolov5l6')

Using cache found in C:\Users\heman/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-25 Python-3.12.2 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)

Fusing layers... 
YOLOv5l6 summary: 476 layers, 76726332 parameters, 0 gradients, 111.2 GFLOPs
Adding AutoShape... 


In [22]:
cap=cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame=cap.read()
    results=model(frame)
    cv2.imshow('MODEL CHECK', np.squeeze(results.render()))

    if cv2.waitKey(10) & 0xFF == ord('f'):
        break
cap.release()
cv2.destroyAllWindows()

In [4]:
from torch.utils.data import Dataset

dataset_folder = '/Users/heman/OneDrive/Documents/GitHub/ELC2024/'

class DroneSignalsDatasetIQandSpec(Dataset):
    
    def __init__(self, x_iq_tensor, x_spec_tensor, y_tensor, snr_tensor, duty_cycle_tensor):
        self.x_iq = x_iq_tensor
        self.x_spec = x_spec_tensor
        self.y = y_tensor
        self.snr = snr_tensor
        self.dury_cyle = duty_cycle_tensor

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x_iq[idx], self.x_spec[idx], self.y[idx], self.snr[idx], self.dury_cyle[idx]   

    def targets(self):
        return self.y 

    def snrs(self):
        return self.snr

    def duty_cycle(self):
        return self.duty_cycle


def plot_input_data(spectrogram_2d, iq_2d, title='', figsize=(10,9)):

    fig, axs = plt.subplot_mosaic([['spec_re', 'spec_im'], ['spec_re', 'spec_im'], ['iq_re', 'iq_re'], ['iq_im', 'iq_im']], figsize=figsize) # layout='constrained'
    
    # plot spectrogram Re and Im
    spec_re = axs['spec_re'].imshow(spectrogram_2d[0,:,:]) #, aspect='auto', origin='lower')
    axs['spec_re'].set_title('Re', fontsize=10)
    fig.colorbar(spec_re, ax=axs['spec_re'], location='right', shrink=0.5)

    spec_im = axs['spec_im'].imshow(spectrogram_2d[1,:,:]) #, aspect='auto', origin='lower')
    axs['spec_im'].set_title('Im', fontsize=10)
    fig.colorbar(spec_im, ax=axs['spec_im'], location='right', shrink=0.5)

    # plot iq Re and Im
    axs['iq_re'].plot(iq_2d[0,:])
    axs['iq_re'].set_title('IQ data')
    axs['iq_re'].set_ylabel('Re', rotation=0)

    axs['iq_im'].plot(iq_2d[1,:])
    # axs['iq_im'].set_title('Im')
    axs['iq_im'].set_xlabel('Time (samples)')
    axs['iq_im'].set_ylabel('Im', rotation=0)
    
    # add figure title
    fig.suptitle(plt_title + '\n\nSpectrogram')
    plt.savefig('sample_input_data.png', dpi=300, bbox_inches='tight')   
    plt.show()


# read statistics/class count of the dataset
dataset_stats = pd.read_csv(dataset_folder + 'class_stats.csv', index_col=0)
class_names = dataset_stats['class'].values

# read SNR count of the dataset
snr_stats = pd.read_csv(dataset_folder + 'SNR_stats.csv', index_col=0)
snr_list = snr_stats['SNR'].values

# load data
dataset_dict = torch.load(dataset_folder + 'dataset.pt')
dataset_dict.keys()

x_iq = dataset_dict['x_iq']
x_spec = dataset_dict['x_spec']
y = dataset_dict['y']
snrs = dataset_dict['snr']
duty_cycle = dataset_dict['duty_cycle']

# create pytorch dataset form tensors
dataset = DroneSignalsDatasetIQandSpec(x_iq,x_spec,y,snrs,duty_cycle)
del(x_iq, x_spec, y, snrs, duty_cycle, dataset_dict)

# define a data loader
data_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=64)

print('Dataset loaded to torch data_loader.')

# get a batch of samples from the data loader
x_iq, x_spec, labels, snrs, duty_cycle =  next(iter(data_loader))
x_iq.shape, x_spec.shape, labels.shape, snrs.shape, duty_cycle.shape

print('Loaded a batch of samples from the data loader with batch size', x_iq.shape[0], 'and the following shapes:')
print('x_iq shape: ', x_iq.shape)
print('x_spec shape: ', x_spec.shape)
print('labels shape: ', labels.shape)
print('snrs shape: ', snrs.shape)
print('duty_cycle shape: ', duty_cycle.shape)



Dataset loaded to torch data_loader.
Loaded a batch of samples from the data loader with batch size 64 and the following shapes:
x_iq shape:  torch.Size([64, 2, 16384])
x_spec shape:  torch.Size([64, 2, 128, 128])
labels shape:  torch.Size([64])
snrs shape:  torch.Size([64])
duty_cycle shape:  torch.Size([64])


In [5]:
# select a sample from the batch
sample_id = 52

# plot the sample
act_snr = snrs[sample_id]
act_duty_cycle = duty_cycle[sample_id]
act_class = class_names[labels[sample_id]]
plt_title = 'Class: ' + act_class + ', SNR: ' + str(act_snr.numpy()) + 'dB, Duty Cycle: ' + str(act_duty_cycle.numpy())

spectrogram_2d = x_spec[sample_id,:,:,:]
iq_2d = x_iq[sample_id,:,:]
plot_input_data(spectrogram_2d, iq_2d, title=plt_title, figsize=(10,7))


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.fft import fft

# Function to extract features from spectrograms
def extract_spectrogram_features(spectrogram):
    mean_val = np.mean(spectrogram)
    median_val = np.median(spectrogram)
    std_val = np.std(spectrogram)
    skewness = skew(spectrogram.flatten())
    kurt = kurtosis(spectrogram.flatten())
    return [mean_val, median_val, std_val, skewness, kurt]

def extract_iq_features(iq_data):
    # Assuming iq_data is a 2D array with shape (2, N)
    iq_features = []

    for channel in iq_data:
        mean_val = np.mean(channel)
        median_val = np.median(channel)
        std_val = np.std(channel)
        skewness = skew(channel)
        kurt = kurtosis(channel)

        # Frequency domain features
        freq_domain = fft(channel)
        dominant_freq = np.argmax(np.abs(freq_domain))
        freq_variance = np.var(np.abs(freq_domain))

        iq_features.extend([mean_val, median_val, std_val, skewness, kurt, dominant_freq, freq_variance])

    return iq_features

# Function to create a DataFrame with extracted features
def create_feature_dataframe(x_iq, x_spec, y, snrs, duty_cycle):
    features = []
    for iq, spec, label, snr, duty in zip(x_iq, x_spec, y, snrs, duty_cycle):
        iq = iq.numpy()
        spec = spec.numpy()
        spectrogram_features = extract_spectrogram_features(spec)
        iq_features = extract_iq_features(iq)
        features.append(spectrogram_features + iq_features + [snr.item(), duty.item(), label.item()])

    columns = ['spec_mean', 'spec_median', 'spec_std', 'spec_skew', 'spec_kurt',
               'iq_mean_ch1', 'iq_median_ch1', 'iq_std_ch1', 'iq_skew_ch1', 'iq_kurt_ch1', 'iq_dom_freq_ch1', 'iq_freq_var_ch1',
               'iq_mean_ch2', 'iq_median_ch2', 'iq_std_ch2', 'iq_skew_ch2', 'iq_kurt_ch2', 'iq_dom_freq_ch2', 'iq_freq_var_ch2',
               'snr', 'duty_cycle', 'label']
    df = pd.DataFrame(features, columns=columns)
    return df

# Extracting features and creating DataFrame
feature_df = create_feature_dataframe(x_iq, x_spec, y, snrs, duty_cycle)

# Save to CSV
feature_df.to_csv('feature_data.csv', index=False)

NameError: name 'y' is not defined